### pydantic

In [2]:

### GROQ model integration
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
model = init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f1ed013f470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1e93b9bcb0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the Movie")
    year:int=Field(description="This year movie was released")
    director:str=Field(description="The character of the movie")
    rating:float=Field(description="The movie rating out of 10")


In [19]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [4]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f1ed013f470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1e93b9bcb0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the Movie', 'type': 'string'}, 'year': {'description': 'This year movie was released', 'type': 'integer'}, 'director': {'description': 'The character of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', 'director', '

In [7]:
response=model_with_structure.invoke("Provide details about movie inception")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message output alongside parsed structure

In [8]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    """A movie with details"""
    title:str=Field(...,description="The title of the Movie")
    year:int=Field(...,description="This year movie was released")
    director:str=Field(...,description="The character of the movie")
    rating:float=Field(...,description="The movie rating out of 10")

model_with_structure=model.with_structured_output(Movie,include_raw=True)
response=model_with_structure.invoke("Provide details about movie inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie "Inception". Let me see. I need to use the Movie function provided. The required parameters are title, year, director, and rating.\n\nFirst, the title is "Inception". The director is Christopher Nolan. I think the release year was 2010. The rating... maybe around 8.8 on IMDb. Let me confirm these details. Yes, that\'s correct. So I\'ll structure the function call with those parameters.\n', 'tool_calls': [{'id': 's4tq91vtk', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 227, 'total_tokens': 378, 'completion_time': 0.253069252, 'completion_tokens_details': {'reasoning_tokens': 103}, 'prompt_time': 0.02176128, 'prompt_tokens_details': None, 'queue_time': 0.05680145, 'total_time': 0.27483053

### Nested structure

In [9]:
from pydantic import BaseModel,Field

class Actor(BaseModel):
    name:str
    role:str

class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure=model.with_structured_output(MovieDetails)
response=model_with_structure.invoke("Provide details about movie inception")
response

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne')], genres=['Science Fiction', 'Action', 'Thriller'], budget=160.0)

## TypedDict

In [12]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A movie with details"""
    title:Annotated[str,...,"The title of the Movie"]
    year:Annotated[str,...,"This year movie was released"]
    director:Annotated[str,...,"The character of the movie"]
    rating:Annotated[float,...,"The movie rating out of 10"]

model_with_typeddict=model.with_structured_output(MovieDict)
response=model_with_typeddict.invoke("Provide details about movie avangers")
response

{'director': 'Joss Whedon',
 'rating': 8,
 'title': 'The Avengers',
 'year': '2012'}

In [13]:

class Actor(TypedDict):
    name:str
    role:str

class MovieDetails(TypedDict):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
    budget:float|None = Field(None,description="Budget in millions USD")


model_with_structure=model.with_structured_output(MovieDetails)
response=model_with_structure.invoke("Provide details about movie inception")
response

{'budget': 160000000,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Ellen Page', 'role': 'Emily'},
  {'name': 'Tom Hardy', 'role': 'John Reilly'}],
 'genres': ['Science Fiction', 'Action', 'Thriller'],
 'title': 'Inception',
 'year': 2010}

In [14]:
model_with_typeddict

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f1ed013f470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1e93b9bcb0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'MovieDict', 'description': 'A movie with details', 'parameters': {'type': 'object', 'properties': {'title': {'description': 'The title of the Movie', 'type': 'string'}, 'year': {'description': 'This year movie was released', 'type': 'string'}, 'director': {'description': 'The character of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'r

In [15]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

## Data classes

In [20]:
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name:str=Field(description="The name of the person")
    email:str=Field(description="This email of the person")
    phone:str=Field(description="The phone of the person")

agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo # auto select Provider Strategy
)

result=agent.invoke({
    "messages":[{"role":"user","content":"Extract cobtact info from : John Doe, json@example.com, (555) 123-7867"}]
})

print(result["structured_response"])

name='John Doe' email='json@example.com' phone='(555) 123-7867'


In [21]:
result

{'messages': [HumanMessage(content='Extract cobtact info from : John Doe, json@example.com, (555) 123-7867', additional_kwargs={}, response_metadata={}, id='fa881709-c494-46e5-9ee7-27c3dd0ff05a'),
  AIMessage(content='{"name":"John Doe","email":"json@example.com","phone":"(555) 123-7867"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 292, 'prompt_tokens': 197, 'total_tokens': 489, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CvenCrQQ88VlIYdDhd4v89LjZAr3l', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9c85-4005-7f43-a9d1-a2b0f0d51062-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 197, 'outpu

## Agent with Typed Dict

In [22]:
from langchain.agents import create_agent
from typing_extensions import TypedDict

class ContactInfo(TypedDict):
    name:str
    email:str
    phone:str

agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo # auto select Provider Strategy
)

result=agent.invoke({
    "messages":[{"role":"user","content":"Extract cobtact info from : John Doe, json@example.com, (555) 123-7867"}]
})

print(result["structured_response"])

{'name': 'John Doe', 'email': 'json@example.com', 'phone': '(555) 123-7867'}


## Data classs example


In [24]:
# data classes

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """Contact information of the person"""
    name:str
    email:str
    phone:str

agent = create_agent(
    model="gpt-5",
    response_format=ContactInfo # auto select Provider Strategy
)

result=agent.invoke({
    "messages":[{"role":"user","content":"Extract cobtact info from : John Doe, json@example.com, (555) 123-7867"}]
})
result
print(result["structured_response"])


ContactInfo(name='John Doe', email='json@example.com', phone='(555) 123-7867')
